# Demo for uproot

In [2]:
import os
import numpy as np
import uproot
import awkward as ak

num_events = 50000 #must be match to the existed root files name
data_path = os.path.expanduser("~/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/")
channels = [
    "ZprimeToZhToZinvhbb",
    "ZprimeToZhToZlephbb",
    "QCD_HT1500to2000",
    "QCD_HT2000toInf",
]

def get_events(idx):
    print(channels[idx])
    channel = channels[idx]
    root_file = f"{data_path}{channel}_{num_events}.root"
    events = uproot.open(root_file+f":jets/Events")
    return events

### Cut by Pt

In [4]:
events = get_events(1)
cut_events = events.arrays(
    expressions = ["fatjet_pt", "fatjet_eta", "fatjet_phi", "fatjet_tau1", "fatjet_daughter_pt"],
    cut = "(fatjet_pt >= 800) & (fatjet_pt <= 1200)"
)

cut_events = cut_events[ak.num(cut_events)["fatjet_pt"] > 0]
print(f"Number of events remain = {len(cut_events)}")

ZprimeToZhToZlephbb
Number of events remain = 10319


### Select the jet/fatjet with max Pt

https://github.com/scikit-hep/awkward/discussions/1906#discussioncomment-4207964

In [5]:
# demo advanced index

x1 = ak.Array([
    [1,2,3],
    [4,5,6,7],
    [8,9],
])

x2 = ak.Array([
    [[-1,2], [3,4], [5,6]],
    [[1,1], [2,2], [3,3], [4,4]],
    [[1,2,3], [4,5,6]],
])

ix = ak.Array([
    [2],
    [0],
    [1]
])

# ak.flatten(x1[ix], axis=1)
x1[ix]

<Array [[3], [4], [9]] type='3 * var * int64'>

In [6]:
events = get_events(3)
jet_type = "fatjet"
events = events.arrays(filter_name=f"{jet_type}*")
max_index = ak.firsts(ak.argsort(events[f"{jet_type}_pt"], ascending=False))
max_index = ak.unflatten(max_index, counts=ak.ones_like(max_index))
events = events[max_index]
for i in range(5):
    print(events["fatjet_pt"][i], events["fatjet_daughter_pdgid"][i])

QCD_HT2000toInf
[1.19e+03] [[2.96e-43, nan, 1.82e-43, 3.08e-44, ..., 3.08e-44, 2.96e-43, nan, nan]]
[913] [[2.96e-43, nan, 2.96e-43, 3.08e-44, ..., 3.08e-44, 3.08e-44, 2.96e-43]]
[906] [[2.96e-43, nan, nan, 2.96e-43, nan, ..., 3.08e-44, 2.96e-43, 3.08e-44, nan]]
[1.54e+03] [[2.96e-43, nan, nan, 2.96e-43, nan, ..., 3.08e-44, 3.08e-44, nan, 2.96e-43]]
[1.32e+03] [[nan, 2.96e-43, 2.96e-43, nan, nan, ..., 3.08e-44, nan, 3.08e-44, 3.08e-44]]
